<a href="https://colab.research.google.com/github/NajmehNyr/Social_Status/blob/main/Preprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd; pd.set_option('future.no_silent_downcasting', True)

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1HiCx_IS2t0PLg1Blmw5XmmFyXS6oQl3H/export?format=csv'
df = pd.read_csv(url)

df.head()

,ID,GENDER,MARR,RELIBIL,FAMINC,PERSINC,RESID,FATEDU,UNNES,ENTER,...,SOCMED,FRI,science,politics,culture,religon,others,nothing,FAMCOM,EDUCOM
0,1,1,1,2,30,0,3,3,۵۰ الی ۷۰ درصد,۱ الی ۳ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,علمی,NaN,فرهنگی,NaN,NaN,NaN,1,3
1,2,1,1,3,60,0,4,2,۳۰ الی ۵۰ درصد,۱ الی ۳ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,NaN,NaN,فرهنگی,NaN,NaN,NaN,2,4
2,3,1,1,3,15,0,2,1,۳۰ الی ۵۰ درصد,زیر ۱ میلیون تومان,...,بالای ۵ ساعت,زیاد,NaN,NaN,NaN,NaN,NaN,هیچکدام,1,2
3,4,1,1,2,17.5,0,3,1,۷۰ الی ۱۰۰ درصد,زیر ۱ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,علمی,NaN,NaN,NaN,NaN,NaN,2,3
4,5,1,1,3,14,0,3,3,۵۰ الی ۷۰ درصد,زیر ۱ میلیون تومان,...,۱ تا ۳ ساعت,زیاد,NaN,NaN,NaN,NaN,سایر,NaN,1,2


#Preprocessing

## Correcting Data Collector's Mistakes

**Replaceing values with new ones**

Here, I will correct some of the data collector's mistakes, so having a bigger number on the scale means better performance in that task. At the same time, I will also convert my strings to numerical values, especially since they are Persian.

In [ ]:
# in 2 first two: sort unique values, then give them ordinal numbers that increase by 1
REPs = {
    'UNNES':  {ValUnn: IndUnn + 1 for IndUnn, ValUnn in enumerate(np.sort(df['UNNES'].unique()))},
    'ENTER':  {ValEnt: IndEnt + 1 for IndEnt, ValEnt in enumerate(np.sort(df['ENTER'].unique()))},
    'READ':   {'زیر یکساعت': 1, 'یک تا سه ساعت': 2, 'سه تا هفت': 3, 'بالای هفت ساعت...': 4},
    'SPORT':  {'خیر ، فاقد مهارت ورزشی': 1, 'در حد مبتدی': 2, 'در حد متوسط': 3, 'بله ، در حد عالی': 4},
    'ENG':    {'فاقد آشنایی': 1, 'در حد کم': 2, 'متوسط': 3, 'زیاد': 4, 'عالی': 5},
    'VOUL':   {'خیلی ضعیف': 1, 'ضعیف': 2, 'متوسط': 3, 'خوب': 4},
    'SOCMED': {'زیر ۱ ساعت': 1, '۱ تا ۳ ساعت': 2, '۳ تا ۵ ساعت': 3, 'بالای ۵ ساعت': 4},
    'FRI':    {'خیلی کم': 1, 'کم': 2, 'متوسط': 3, 'زیاد': 4},
    'FAMCOM': {1: 3, 3: 1},
    'RESID':  {1: 5, 2: 4, 4: 2, 5: 1}}
df = df.replace(REPs)


**Combining and Dropping Some of The Columns**

There are six columns related to the type of NGOs individuals are involved in.

`NGO_Types = ['science', 'culture', 'religion', 'politics', 'others', 'nothing']`.
I will count involvement in each of these columns, except for `NGO_Types['nothing']`, and then create a column named "NGOs" for them.

The nothing column isn't needed here, so I will drop it alongside marriage status since none of the individuals is married in our survey.

In [ ]:
# NGO Envolvement count
NGOs = ['science', 'culture', 'religon', 'politics', 'others']
for NGO in NGOs: #fill NaNs with 0 and non-NANs with 1
  df[NGO] = df[NGO].fillna(0).apply(lambda x: 1 if x != 0 else 0)
df['NGONUM'] = df[NGOs].sum(axis=1)

# Drop columns that are useless or not needed anymore
df.drop(['nothing', 'MARR'] + NGOs, axis=1, inplace=True)

## Handling Missing Data

After I turned all my data points to numeric values I search for missing data points and fill those with appropriate values.

Columns with missing data:
*  Family Income: Median due to that variability.
*  Person's Income: 0 because most of the individuals didn't make any money, so if they didn't answer this, they probably were the same.


In [ ]:
# Convert columns to numeric and errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Fill missing values
df['FAMINC'] = df['FAMINC'].fillna(df['FAMINC'].median())
df.at[30, "PERSINC"] = 0
df['COMP'] = df['COMP'].fillna(df['COMP'].mean()).astype(int)

In [ ]:
# Check if there are any null values left
NullLen = df.isnull().sum()
NullCol = NullLen[NullLen > 0]
if not NullCol.empty: print(f"Columns with remaining null values: {NullCol}")

## Converting All the Values to Scores

**Income varibles**: We ask people how much they make per month (million Rial) but those valuses need to be transfered to scores.

**Personal Income**: Only a minority had thier own outcome we have 3 scores for Personal income and 5 for Family Income.

In [ ]:
# specify the cuts, for FAMINC create 5 cuts and 3 for PERSINC
FamincCuts = [np.min(df['FAMINC']) + faminc_i * (np.ptp(df['FAMINC']) / 5) for faminc_i in range(6)]
FamincCuts[-1] = np.max(df['FAMINC']) + 1 # just for fixing a minor error
PersincCuts = [0, 5, 10, 51]

# Label the cuts
df['FAMINC_SCORE'] = pd.cut(df['FAMINC'], bins=FamincCuts, labels=[1, 2, 3, 4, 5], right=False, include_lowest=True)
df['PERSINC_SCORE'] = pd.cut(df['PERSINC'], bins=PersincCuts, labels=[1, 2, 3], right=False, include_lowest=True)

df['FAMINC_SCORE'] = df['FAMINC_SCORE'].astype(int)
df['PERSINC_SCORE'] = df['PERSINC_SCORE'].astype(int)

## Creating Our Main Variables

In this project, we aim to measure the classes of economic, social, and cultural scores to compare each person's score to others.
For that, we should create economic, social, cultural, socio-cultural, and overall scores.

In [ ]:
# Create main variables dict and get their mean
vars= {
    'ECO': {'FAMINC_SCORE', 'PERSINC_SCORE', 'RESID' , 'UNNES' , 'ENTER' ,  'JOBEXP'       },
    'CUL': {'GENDER'      , 'RELIBIL'      , 'FATEDU', 'READ'  , 'SPORT' ,  'COMP'  , 'ENG'},
    'SOC': {'VOUL'        , 'SOCMED'       , 'FRI'   , 'NGONUM', 'FAMCOM',  'EDUCOM'       }}
for Var, SubVar in vars.items():
  df[Var] = df[list(SubVar)].mean(axis=1)

In [ ]:
# Create Additional variables
df['SOCCUL'] = df[['CUL', 'SOC']].mean(axis=1)
df['OVERALL'] = df[['ECO', 'CUL', 'SOC']].mean(axis=1)
ADDITIONAL = ['SOCCUL', 'OVERALL']

# Rearrange the data[ID and INCS + vars[subvars] + list(vars) + ADDITIONAL]
NewOrder = ['ID', 'PERSINC', 'FAMINC'] + [subvar for var in vars.keys() for subvar in list(vars[var])] + list(vars.keys()) + ADDITIONAL
df = df[NewOrder]

## Calculate min and max

The theoretical minimum and maximum are different from the ones participants actually choose, so I calculated the theoretical ones for normalization.

In [ ]:
# Write min and max based on survey options
MinMax = {'GENDER':[1,2], 'RELIBIL':[1,5], 'SPORT':[1,4]      , 'RESID':[1,5]  ,
          'FATEDU':[1,5],'UNNES':[1,4]   , 'ENTER':[1,4]       , 'JOBEXP':[1,3]      , 'READ':[1,4]   ,
          'COMP':[1,4]  , 'ENG':[1,5]    , 'VOUL':[1,5]        ,'SOCMED':[1,4]       , 'FRI':[1,5]    ,
          'EDUCOM':[1,5], 'NGONUM':[0,5] , 'FAMINC_SCORE':[1,5],'PERSINC_SCORE':[1,3],  'FAMCOM':[1,3],
          'FAMINC':[df['FAMINC'].min(), df['FAMINC'].max()],
          'PERSINC':[df['PERSINC'].min(), df['PERSINC'].max()]}

In [ ]:
# 1: Calculate min and max based on minmax dict

# For Vars
VarMin = [MinMax[SubVarm][0] for SubVarm in SubVar]
VarMax = [MinMax[SubVarm][1] for SubVarm in SubVar]
MinMaxVars = {Var: (np.mean(VarMin), np.mean(VarMax))
              for Var, SubVar in vars.items()}

# For ADDITIONALs
SOCCULMin = [MinMaxVars['SOC'][0], MinMaxVars['CUL'][0]]
SOCCULMax = [MinMaxVars['SOC'][1], MinMaxVars['CUL'][1]]

OVERALLMin = [MinMaxVars['ECO'][0], MinMaxVars['CUL'][0], MinMaxVars['SOC'][0]]
OVERALLMax = [MinMaxVars['ECO'][1], MinMaxVars['CUL'][1], MinMaxVars['SOC'][1]]

MinMaxAdd = {'SOCCUL': (np.mean(SOCCULMin), np.mean(SOCCULMax)),
             'OVERALL': (np.mean(OVERALLMin), np.mean(OVERALLMax))}


# 2: Update the MinMax dictionary with var means and additional means
MinMax.update(MinMaxVars)
MinMax.update(MinMaxAdd)
df.head()

,ID,PERSINC,FAMINC,RESID,PERSINC_SCORE,FAMINC_SCORE,UNNES,ENTER,JOBEXP,GENDER,...,VOUL,FRI,NGONUM,EDUCOM,FAMCOM,ECO,CUL,SOC,SOCCUL,OVERALL
0,1,0.0,30.0,3,1,2,3,3,1,1,...,3,3,2,3,3,2.166667,2.285714,2.666667,2.476190,2.373016
1,2,0.0,60.0,2,1,3,2,3,1,1,...,4,3,1,4,2,2.000000,1.857143,2.666667,2.261905,2.174603
2,3,0.0,15.0,4,1,1,2,2,3,1,...,2,4,0,2,3,2.166667,1.571429,2.500000,2.035714,2.079365
3,4,0.0,17.5,3,1,1,4,2,1,1,...,3,3,1,3,2,2.000000,2.571429,2.333333,2.452381,2.301587
4,5,0.0,14.0,3,1,1,3,2,1,1,...,3,4,1,2,3,1.833333,2.000000,2.500000,2.250000,2.111111


In [ ]:
df.to_csv('preprocessed_social_status.csv', index=False)

# Normalization
**Min-Max scaling**

I use Min-Max scaling because, as I said, there's a difference between theoretical and practical min and max, which itself is important since one in our sample gets more than 0.6 score in economics and probably reflects the high inflation rate and broken economics of Iran.

In addition, Variables didn't use the same scaling. We had 1-2, 1-3, 1-4, 1-5, and it seems reasonable to use min-max scaling, which will also make data interpretation more understandable.

In [ ]:
# Define the min_max scaling fuction

def MMScaling(InpCol):

  Min = MinMax[InpCol.name][0]
  Max = MinMax[InpCol.name][1]
  MMScaled = (InpCol - Min) / (Max - Min)
  return MMScaled

In [ ]:
# Create the Normalized Data Frame(ndf)
ndf = df.copy()

# Identify categories and columns to normalize
NCols = ['ECO', 'CUL', 'SOC', 'SOCCUL', 'OVERALL']
for var in vars.keys():
    NCols.extend(list(vars[var]))

# Convert columns to numeric and normalize them
for col in NCols:
    ndf[col] = pd.to_numeric(ndf[col], errors='coerce')
    ndf[col] = MMScaling(ndf[col])
ndf.head()

,ID,PERSINC,FAMINC,RESID,PERSINC_SCORE,FAMINC_SCORE,UNNES,ENTER,JOBEXP,GENDER,...,VOUL,FRI,NGONUM,EDUCOM,FAMCOM,ECO,CUL,SOC,SOCCUL,OVERALL
0,1,0.0,30.0,0.50,0.0,0.25,0.666667,0.666667,0.0,0.0,...,0.50,0.50,0.4,0.50,1.0,0.363636,0.396104,0.500000,0.448052,0.419913
1,2,0.0,60.0,0.25,0.0,0.50,0.333333,0.666667,0.0,0.0,...,0.75,0.50,0.2,0.75,0.5,0.318182,0.279221,0.500000,0.389610,0.365801
2,3,0.0,15.0,0.75,0.0,0.00,0.333333,0.333333,1.0,0.0,...,0.25,0.75,0.0,0.25,1.0,0.363636,0.201299,0.454545,0.327922,0.339827
3,4,0.0,17.5,0.50,0.0,0.00,1.000000,0.333333,0.0,0.0,...,0.50,0.50,0.2,0.50,0.5,0.318182,0.474026,0.409091,0.441558,0.400433
4,5,0.0,14.0,0.50,0.0,0.00,0.666667,0.333333,0.0,0.0,...,0.50,0.75,0.2,0.25,1.0,0.272727,0.318182,0.454545,0.386364,0.348485


In [ ]:
# Reorder the Normalized Data Frame
ndfOG = ndf.columns
NewOrder2 = ['ID', 'PERSINC_SCORE', 'FAMINC_SCORE', 'ENTER', 'RESID', 'JOBEXP',
       'UNNES', 'GENDER', 'ENG', 'SPORT', 'FATEDU', 'RELIBIL',
       'COMP', 'READ', 'EDUCOM', 'SOCMED', 'FRI', 'NGONUM', 'FAMCOM', 'VOUL',
       'ECO', 'CUL', 'SOC', 'SOCCUL', 'OVERALL']
ndf = ndf[NewOrder2].round(4)

# Rename PERSINC_SCORE and FAMINC_SCORE
ndf.rename(columns={'PERSINC_SCORE': 'PERSINC', 'FAMINC_SCORE': 'FAMINC'}, inplace=True)

# Change PERSINC_SCORE and FAMINC_SCORE names in vars and minmax too
vars['ECO'] = {'FAMINC', 'PERSINC', 'RESID' , 'UNNES' , 'ENTER' ,  'JOBEXP'}
MinMax.pop('FAMINC_SCORE'); MinMax['FAMINC'] = [1,5]
MinMax.pop('PERSINC_SCORE'); MinMax['PERSINC'] = [1,3]
print(vars)
ndf.head()

{'ECO': {'RESID', 'PERSINC', 'UNNES', 'ENTER', 'FAMINC', 'JOBEXP'}, 'CUL': {'GENDER', 'SPORT', 'FATEDU', 'ENG', 'READ', 'COMP', 'RELIBIL'}, 'SOC': {'SOCMED', 'VOUL', 'FRI', 'NGONUM', 'EDUCOM', 'FAMCOM'}}


,ID,PERSINC,FAMINC,ENTER,RESID,JOBEXP,UNNES,GENDER,ENG,SPORT,...,SOCMED,FRI,NGONUM,FAMCOM,VOUL,ECO,CUL,SOC,SOCCUL,OVERALL
0,1,0.0,0.25,0.6667,0.50,0.0,0.6667,0.0,0.50,0.6667,...,0.3333,0.50,0.4,1.0,0.50,0.3636,0.3961,0.5000,0.4481,0.4199
1,2,0.0,0.50,0.6667,0.25,0.0,0.3333,0.0,0.50,0.3333,...,0.3333,0.50,0.2,0.5,0.75,0.3182,0.2792,0.5000,0.3896,0.3658
2,3,0.0,0.00,0.3333,0.75,1.0,0.3333,0.0,0.25,0.3333,...,1.0000,0.75,0.0,1.0,0.25,0.3636,0.2013,0.4545,0.3279,0.3398
3,4,0.0,0.00,0.3333,0.50,0.0,1.0000,0.0,1.00,0.3333,...,0.3333,0.50,0.2,0.5,0.50,0.3182,0.4740,0.4091,0.4416,0.4004
4,5,0.0,0.00,0.3333,0.50,0.0,0.6667,0.0,0.50,0.3333,...,0.3333,0.75,0.2,1.0,0.50,0.2727,0.3182,0.4545,0.3864,0.3485


In [ ]:
if len(df.columns) == len(ndfOG): print('All the columns are in normalized df too.')
if len(ndf.columns) != len(ndfOG): print('Income Scores are removed.')

All the columns are in normalized df too.
Income Scores are removed.


In [ ]:
df.to_csv('normalized_social_status.csv', index=False)